# Data Preprocessing

In [1]:
import os, time, random
import numpy as np
import pandas as pd
import cv2, torch
from tqdm.auto import tqdm
import shutil as sh
from sklearn import preprocessing, model_selection

from IPython.display import Image, clear_output
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [2]:
img_path = 'datasets/TrainImages/'
shapes = []
for img in tqdm(os.listdir(img_path)):
    my_img = cv2.imread(img_path+img)
    shapes.append(my_img.shape)
print(set(shapes))
img_w = 1000
img_h = 1000

  0%|          | 0/660 [00:00<?, ?it/s]

{(1000, 1000, 3)}

- Train

In [3]:
source_lable_path = 'datasets/TrainGroundTruth/'
destination_lable_path = 'labels/'


read_list = []
for lable in tqdm(os.listdir(source_lable_path)):
    with open(source_lable_path+lable, "r") as txt_file:
        xmin,ymin,xmax,ymax = [float(i) for i in txt_file.readlines()[0][:-1].split(',')]
        read_list.append([lable[:-4],xmin,ymin,xmax,ymax])
        
df = pd.DataFrame(read_list,columns=['name','xmin','ymin','xmax','ymax'])
df

  0%|          | 0/660 [00:00<?, ?it/s]

,name,xmin,ymin,xmax,ymax
0,1,509.0,847.0,938.0,903.0
1,10,422.0,662.0,740.0,715.0
2,100,429.0,669.0,551.0,700.0
3,101,532.0,262.0,842.0,320.0
4,102,11.0,736.0,366.0,804.0
...,...,...,...,...,...
655,95,462.0,617.0,789.0,707.0
656,96,543.0,793.0,600.0,834.0
657,97,59.0,713.0,432.0,787.0
658,98,394.0,634.0,505.0,704.0


In [4]:
df['x_center'] = (df['xmin'] + df['xmax'])/2
df['y_center'] = (df['ymin'] + df['ymax'])/2
df['w'] = df['xmax'] - df['xmin']
df['h'] = df['ymax'] - df['ymin']
df['labels'] = 0
df['x_center_norm'] = df['x_center']/img_w
df['y_center_norm'] = df['y_center']/img_h
df['w_norm'] = df['w']/img_w
df['h_norm'] = df['h']/img_h
df

,name,xmin,ymin,xmax,ymax,x_center,y_center,w,h,labels,x_center_norm,y_center_norm,w_norm,h_norm
0,1,509.0,847.0,938.0,903.0,723.5,875.0,429.0,56.0,0,0.7235,0.8750,0.429,0.056
1,10,422.0,662.0,740.0,715.0,581.0,688.5,318.0,53.0,0,0.5810,0.6885,0.318,0.053
2,100,429.0,669.0,551.0,700.0,490.0,684.5,122.0,31.0,0,0.4900,0.6845,0.122,0.031
3,101,532.0,262.0,842.0,320.0,687.0,291.0,310.0,58.0,0,0.6870,0.2910,0.310,0.058
4,102,11.0,736.0,366.0,804.0,188.5,770.0,355.0,68.0,0,0.1885,0.7700,0.355,0.068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,95,462.0,617.0,789.0,707.0,625.5,662.0,327.0,90.0,0,0.6255,0.6620,0.327,0.090
656,96,543.0,793.0,600.0,834.0,571.5,813.5,57.0,41.0,0,0.5715,0.8135,0.057,0.041
657,97,59.0,713.0,432.0,787.0,245.5,750.0,373.0,74.0,0,0.2455,0.7500,0.373,0.074
658,98,394.0,634.0,505.0,704.0,449.5,669.0,111.0,70.0,0,0.4495,0.6690,0.111,0.070


In [5]:
df_train, df_valid = model_selection.train_test_split(df, test_size=0.1, random_state=13, shuffle=True)
print(df_train.shape, df_valid.shape)

(594, 14) (66, 14)


In [6]:
source_img_path = 'datasets/TrainImages/'
destination_img_path = 'datasets/signature/images/train/'
destination_img_path_val = 'datasets/signature/images/val/'
train_label_path = 'datasets/signature/labels/train/'
val_label_path = 'datasets/signature/labels/val/'
for img in tqdm(os.listdir(source_img_path)):
    row = df_train[df_train['name'] == img.split('.')[0]][['labels', 'x_center_norm', 'y_center_norm', 'w_norm', 'h_norm']]
    if len(row):
        sh.copy(source_img_path+img, destination_img_path+img)
        np.savetxt(train_label_path+str(img.split('.')[0])+'.txt',np.array(row),fmt=["%d", "%f", "%f", "%f", "%f"])   
    row = df_valid[df_valid['name'] == img.split('.')[0]][['labels', 'x_center_norm', 'y_center_norm', 'w_norm', 'h_norm']]
    if len(row):
        sh.copy(source_img_path+img, destination_img_path_val+img)
        np.savetxt(val_label_path+str(img.split('.')[0])+'.txt',np.array(row),fmt=["%d", "%f", "%f", "%f", "%f"])   

  0%|          | 0/660 [00:00<?, ?it/s]

In [7]:
source_lable_path = 'datasets/TestGroundTruth/'
destination_lable_path = 'labels/'

read_list = []
for lable in tqdm(os.listdir(source_lable_path)):
    with open(source_lable_path+lable, "r") as txt_file:
        xmin,ymin,xmax,ymax = [float(i) for i in txt_file.readlines()[0][:-1].split(',')]
        read_list.append([lable[:-4],xmin,ymin,xmax,ymax])
        
df = pd.DataFrame(read_list,columns=['name','xmin','ymin','xmax','ymax'])
df

  0%|          | 0/115 [00:00<?, ?it/s]

,name,xmin,ymin,xmax,ymax
0,680,493.0,692.0,698.0,742.0
1,681,138.0,684.0,275.0,714.0
2,682,534.0,465.0,738.0,506.0
3,683,622.0,553.0,771.0,599.0
4,684,442.0,594.0,674.0,674.0
...,...,...,...,...,...
110,796,528.0,285.0,754.0,338.0
111,797,532.0,436.0,847.0,495.0
112,798,557.0,680.0,764.0,754.0
113,799,533.0,322.0,760.0,374.0


In [8]:
df['x_center'] = (df['xmin'] + df['xmax'])/2
df['y_center'] = (df['ymin'] + df['ymax'])/2
df['w'] = df['xmax'] - df['xmin']
df['h'] = df['ymax'] - df['ymin']
df['labels'] = 0
df['x_center_norm'] = df['x_center']/img_w
df['y_center_norm'] = df['y_center']/img_h
df['w_norm'] = df['w']/img_w
df['h_norm'] = df['h']/img_h
df

,name,xmin,ymin,xmax,ymax,x_center,y_center,w,h,labels,x_center_norm,y_center_norm,w_norm,h_norm
0,680,493.0,692.0,698.0,742.0,595.5,717.0,205.0,50.0,0,0.5955,0.7170,0.205,0.050
1,681,138.0,684.0,275.0,714.0,206.5,699.0,137.0,30.0,0,0.2065,0.6990,0.137,0.030
2,682,534.0,465.0,738.0,506.0,636.0,485.5,204.0,41.0,0,0.6360,0.4855,0.204,0.041
3,683,622.0,553.0,771.0,599.0,696.5,576.0,149.0,46.0,0,0.6965,0.5760,0.149,0.046
4,684,442.0,594.0,674.0,674.0,558.0,634.0,232.0,80.0,0,0.5580,0.6340,0.232,0.080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,796,528.0,285.0,754.0,338.0,641.0,311.5,226.0,53.0,0,0.6410,0.3115,0.226,0.053
111,797,532.0,436.0,847.0,495.0,689.5,465.5,315.0,59.0,0,0.6895,0.4655,0.315,0.059
112,798,557.0,680.0,764.0,754.0,660.5,717.0,207.0,74.0,0,0.6605,0.7170,0.207,0.074
113,799,533.0,322.0,760.0,374.0,646.5,348.0,227.0,52.0,0,0.6465,0.3480,0.227,0.052


In [9]:
source_img_path = 'datasets/TestImages/'
destination_img_path = 'datasets/signature/images/test/'
train_label_path = 'datasets/signature/labels/test/'
for img in tqdm(os.listdir(source_img_path)):
    row = df[df['name'] == img.split('.')[0]][['labels', 'x_center_norm', 'y_center_norm', 'w_norm', 'h_norm']]
    if len(row):
        sh.copy(source_img_path+img, destination_img_path+img)
        np.savetxt(train_label_path+str(img.split('.')[0])+'.txt',np.array(row),fmt=["%d", "%f", "%f", "%f", "%f"])   

  0%|          | 0/115 [00:00<?, ?it/s]

### Train Model 

In [1]:
! python yolov5/train.py --img 640 --batch 16 --epochs 50 --data datasets/signture.yaml --weights yolov5/yolov5s.pt --hyp yolov5/data/hyps/hyp.scratch-low.yaml

train: weights=yolov5/yolov5s.pt, cfg=, data=datasets/signture.yaml, hyp=yolov5\data\hyps\hyp.scratch-low.yaml, epochs=1, batch_size=8, imgsz=64, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5\runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 
fatal: cannot change to 'D:\Subjects\2-': No such file or directory
YOLOv5  2022-12-25 Python-3.9.12 torch-1.13.1+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, a

        0/0         0G     0.1734  0.0009691          0         11         64:  65%|######5   | 49/75 [00:32<00:11,  2.24it/s]
        0/0         0G     0.1734  0.0009691          0         11         64:  67%|######6   | 50/75 [00:32<00:10,  2.28it/s]
        0/0         0G     0.1733  0.0009743          0         12         64:  67%|######6   | 50/75 [00:32<00:10,  2.28it/s]
        0/0         0G     0.1733  0.0009743          0         12         64:  68%|######8   | 51/75 [00:32<00:10,  2.34it/s]
        0/0         0G     0.1732  0.0009742          0          9         64:  68%|######8   | 51/75 [00:33<00:10,  2.34it/s]
        0/0         0G     0.1732  0.0009742          0          9         64:  69%|######9   | 52/75 [00:33<00:10,  2.18it/s]
        0/0         0G      0.173   0.000992          0         11         64:  69%|######9   | 52/75 [00:33<00:10,  2.18it/s]
        0/0         0G      0.173   0.000992          0         11         64:  71%|#######   | 53/75 [00:33<00

### Test Model 

In [2]:
! python yolov5/detect.py --img 640 --source datasets/signature/images/test --weights yolov5/runs/train/best_/weights/best.pt --hide-conf --line-thickness 2 --name best_detect_

detect: weights=['yolov5/runs/train/best_/weights/best.pt'], source=datasets/signature/images/test, data=yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=best_detect_, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=True, half=False, dnn=False, vid_stride=1
fatal: cannot change to 'D:\Subjects\2-': No such file or directory
YOLOv5  2022-12-25 Python-3.9.12 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
image 1/115 D:\Subjects\2- Vision\Handwritten-Signature-Identification-and-Verification\datasets\signature\images\test\680.tif: 640x640 1 Signature, 1898.2ms
image 2/115 D:\Subjects\2- Vision\Handwritten-Signature-Identification-and-Verification\datasets\signature\ima

In [3]:
! python yolov5/val.py --batch 16 --data datasets/signture.yaml --task test --weights yolov5/runs/train/best_/weights/best.pt --name best_val_

val: data=datasets/signture.yaml, weights=['yolov5/runs/train/best_/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5\runs\val, name=best_val_, exist_ok=False, half=False, dnn=False
fatal: cannot change to 'D:\Subjects\2-': No such file or directory
YOLOv5  2022-12-25 Python-3.9.12 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs

test: Scanning D:\Subjects\2- Vision\Handwritten-Signature-Identification-and-Verification\datasets\signature\labels\test.cache... 115 images, 0 backgrounds, 0 corrupt: 100%|##########| 115/115 [00:00<?, ?it/s]
test: Scanning D:\Subjects\2- Vision\Handwritten-Signature-Identification-and-Verification\datasets\signature\labels\test.cache... 115 images, 0 backgrounds, 0 corrupt: 100%|##########| 1